In [1]:
!git clone https://github.com/razandakkak/zaka.git
%cd zaka
!pip install -r requirements.txt
!git clone https://github.com/neuralchen/SimSwap
!mkdir -p SimSwap/checkpoints

!mkdir -p SimSwap/insightface_func/models
!mkdir -p SimSwap/parsing_model/checkpoint
!gdown --id 1PsvCnILPVnfXSVplvbtfmYC4a6Wy2mZx
!mv ./79999_iter.pth SimSwap/parsing_model/checkpoint
!gdown --id 1eg5abGyFk-q5PbYiba4LbihwX4rNyEh-
!mv ./antelope.zip SimSwap/insightface_func/models
!unzip ./SimSwap/insightface_func/models/antelope.zip -d ./SimSwap/insightface_func/models/
!mkdir -p SimSwap/arcface_model
!gdown --id 1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N -O SimSwap/arcface_model/arcface_checkpoint.tar
!mkdir ./SimSwap/checkpoints/people
!gdown --id 1-NwyIz9prIuRD1HSnLHxlxCSLP_1y5EK
!mv ./latest_net_G.pth ./SimSwap/checkpoints/people

!pip install tts torch ffmpeg-python
!pip install pydub
!apt-get install -y ffmpeg

import locale
locale.getpreferredencoding = lambda: 'UTF-8'
!pip install gradio

Cloning into 'zaka'...
remote: Enumerating objects: 297, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 297 (delta 55), reused 0 (delta 0), pack-reused 182 (from 1)
Receiving objects: 100% (297/297), 51.95 MiB | 17.12 MiB/s, done.
Resolving deltas: 100% (122/122), done.
/content/zaka
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 33.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 76.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ...

Cloning into 'SimSwap'...
remote: Enumerating objects: 1141, done.
remote: Counting objects: 100% (1141/1141), done.
remote: Compressing objects: 100% (557/557), done.
remote: Total 1141 (delta 603), reused 1015 (delta 566), pack-reused 0 (from 0)
Receiving objects: 100% (1141/1141), 211.46 MiB | 26.22 MiB/s, done.
Resolving deltas: 100% (603/603), done.
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1PsvCnILPVnfXSVplvbtfmYC4a6Wy2mZx
To: /content/zaka/79999_iter.pth
100% 53.3M/53.3M [00:01<00:00, 30.6MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://d

In [3]:
!ls

CheckCuda.py  Flaskapp.py	      guidance	README.md	  SimSwap    TestGender.py
female	      GenderModel_YOLOv10.pt  male	requirements.txt  templates


In [2]:
import gradio as gr
from PIL import Image
from ultralytics import YOLO
import os
import subprocess
import ffmpeg
from TTS.api import TTS

# Load the pre-trained YOLO gender detection model
gender_model_path = './GenderModel_YOLOv10.pt'
gender_model = YOLO(gender_model_path)

# Define paths for male and female videos
male_videos = "/content/zaka/male"
female_videos = "/content/zaka/female"
output_video_path = "/content/zaka/SimSwap/output/output_video.mp4"  # Path where SimSwap video will be saved

# Actor face mapping
video_to_actor_map = {
    "/content/zaka/female/MaguyBouGhoson.mp4": "/content/zaka/female/Maguyy.png",
    "/content/zaka/female/Iman.mp4": "/content/zaka/female/ImanSayed.png",
    "/content/zaka/female/SamiAlJazaeri.mp4": "/content/zaka/female/samia.png",
    "/content/zaka/female/Shokran.mp4": "/content/zaka/female/Torfa.png",
    "/content/zaka/male/KousayKhawli.mp4": "/content/zaka/male/Kousay.png",
    "/content/zaka/male/Joude.mp4": "/content/zaka/male/Day3aDay3a.png",
    "/content/zaka/male/Moetassem.mp4": "/content/zaka/male/Moetassem.png",
    "/content/zaka/male/Taym AL Hassan.mp4": "/content/zaka/male/TaymAlHassan.png",
    "/content/zaka/male/Mohamad Henedi.mp4": "/content/zaka/male/MohammadHnedi.png"
}

# Function to get the list of videos based on gender
def get_videos_by_gender(gender):
    if gender == 'male':
        return [f for f in os.listdir(male_videos) if f.endswith(".mp4")]
    elif gender == 'female':
        return [f for f in os.listdir(female_videos) if f.endswith(".mp4")]
    return []

# Gender Detection and Video Listing Function
def detect_gender_and_show_videos(image):
    image = Image.open(image)  # Open the uploaded image
    results = gender_model.predict(image, verbose=False)  # Predict using YOLO model

    confidence_threshold = 0.5  # Confidence threshold for predictions
    detected_gender = None

    # Process detection results
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls)
            confidence = box.conf.item()
            if confidence >= confidence_threshold:
                detected_gender = 'female' if class_id == 1 else 'male'

    if detected_gender is None:
        return "Gender not detected. Try again with a clearer image.", gr.update(choices=[])

    # Get list of videos based on detected gender
    videos = get_videos_by_gender(detected_gender)

    if videos:
        return f"Detected gender: {detected_gender}", gr.update(choices=videos, value=videos[0])
    else:
        return f"Detected gender: {detected_gender}. No videos available.", gr.update(choices=[])

# Function to run SimSwap based on selected video
def run_simswap(image, video):
    user_face_path = image
    selected_video = os.path.join(male_videos, video) if video in os.listdir(male_videos) else os.path.join(female_videos, video)
    actor_face = video_to_actor_map[selected_video]

    output_video_path = "/content/zaka/SimSwap/output/output_video.mp4"

    # Command to run SimSwap
    command = f"""
    python test_video_swapspecific.py --no_simswaplogo --crop_size 224 --use_mask \\
    --pic_specific_path "{actor_face}" \\
    --name people \\
    --Arc_path arcface_model/arcface_checkpoint.tar \\
    --pic_a_path "{user_face_path}" \\
    --video_path "{selected_video}" \\
    --checkpoints_dir "./checkpoints" \\
    --output_path "{output_video_path}" \\
    --temp_path ./temp_results
    """

    # Navigate into SimSwap directory and run command
    os.chdir('SimSwap')
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    if process.returncode == 0:
        return output_video_path  # Return the video file path for Gradio to use later
    else:
        return f"Error occurred during SimSwap execution: {stderr.decode()}"

# Function to convert the uploaded audio file to .wav
def convert_audio_to_wav(audio):
    output_audio_path = "converted_audio.wav"  # Path for the output .wav file

    try:
        # Convert the input audio to .wav using ffmpeg
        ffmpeg.input(audio).output(output_audio_path).run(overwrite_output=True)
        return output_audio_path  # Return the path of the converted file
    except Exception as e:
        return f"Error in conversion: {str(e)}"

# Function to extract audio from video and save it as .wav
def extract_audio_from_video(video):
    output_audio_path = "extracted_audio.wav"  # Path for the extracted audio file

    try:
        # Extract audio from video and save as .wav using ffmpeg
        ffmpeg.input(video).output(output_audio_path).run(overwrite_output=True)
        return output_audio_path  # Return the path of the extracted audio
    except Exception as e:
        return f"Error in extraction: {str(e)}"

# Function to clone the user's voice onto the extracted audio
def clone_voice_and_replace_audio(audio, video):
    # Step 1: Convert audio to .wav
    converted_audio = convert_audio_to_wav(audio)

    # Step 2: Extract audio from video
    extracted_audio = extract_audio_from_video(video)

    output_cloned_audio_path = "cloned_audio.wav"  # Path for the cloned audio
    output_video_with_cloned_audio = "video_with_cloned_audio.mp4"  # Path for the output video

    # Initialize the TTS model for voice cloning
    tts = TTS(model_name="voice_conversion_models/multilingual/vctk/freevc24", progress_bar=False)

    try:
        # Perform voice conversion using the extracted audio and user's converted voice
        tts.voice_conversion_to_file(extracted_audio, converted_audio, output_cloned_audio_path)

        # Replace the audio in the video with the cloned audio
        os.system(f"""
            ffmpeg -i {video} -i {output_cloned_audio_path} \\
            -c:v copy -c:a aac -strict experimental \\
            -map 0:v:0 -map 1:a:0 {output_video_with_cloned_audio}
        """)
        return output_video_with_cloned_audio  # Return the path of the output video
    except Exception as e:
        return f"Error in cloning voice or replacing audio: {str(e)}"

# Gradio Interface
def gradio_interface():
    with gr.Blocks() as iface:
        # Upload the image for gender detection
        with gr.Row():
            image_input = gr.Image(type="filepath", label="Upload your image")
            video_output = gr.Dropdown(label="Select a Video", choices=[], interactive=True)

        # Button to run SimSwap after video selection
        swap_button = gr.Button("Run SimSwap")

        # Text output for detected gender
        gender_text = gr.Textbox(label="Gender")

        # Video output for displaying the result
        result_video = gr.Video()

        # Detect gender and show videos when image is uploaded
        image_input.change(fn=detect_gender_and_show_videos, inputs=image_input, outputs=[gender_text, video_output])

        # Run SimSwap when the button is pressed
        swap_button.click(fn=run_simswap, inputs=[image_input, video_output], outputs=result_video)

        # Section to upload audio for voice cloning
        with gr.Row():
            audio_input = gr.Audio(type="filepath", label="Upload your audio file")
            clone_button = gr.Button("Clone Voice and Replace Audio")
            final_video_output = gr.Video(label="Final Video with Cloned Voice")

        # Clone the voice and replace audio in the SimSwap video
        clone_button.click(fn=clone_voice_and_replace_audio, inputs=[audio_input, result_video], outputs=final_video_output)
    iface.launch()

# Launch the Gradio Interface
gradio_interface()


/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:775: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b5ee2ea69b8bb20491.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
